In [1]:
# Import local files
from manage_path import *
from get_data import *
from compute_lda import *

/cliphomes/amao1/miniconda3/envs/finra/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable


In [2]:
data = load_pickle(file_name="FINRA_TRACE_2014.pkl.zip")

Getting data from/cliphomes/amao1/FINRA_TRACE/Data/Pickle/FINRA_TRACE_2014.pkl.zip...
Data getting success from /cliphomes/amao1/FINRA_TRACE/Data/Pickle/FINRA_TRACE_2014.pkl.zip!


In [3]:
data.dtypes

BOND_SYM_ID                    object
CUSIP_ID                       object
SCRTY_TYPE_CD                  object
ENTRD_VOL_QT                  float64
RPTD_PR                       float64
RPT_SIDE_CD                    object
Year                          float64
document_date                  object
TRD_EXCTN_DTTM         datetime64[ns]
TRD_RPT_DTTM           datetime64[ns]
Report_Dealer_Index            object
Contra_Party_Index             object
TRC_ST                         object
RATING_MR                      object
RATING_MR_Numeric             float64
OFFERING_DATE          datetime64[ns]
ISSUE_ID                      float64
MATURITY               datetime64[ns]
OFFERING_AMT                  float64
AMOUNT_OUTSTANDING            float64
Vol_grt_out                   float64
dtype: object

In [9]:
%%time
bag_of_words = trade_SV_BoW(data)
# bag_of_words = trade_vol_BoW(data,"large")

creating documents ......
documents created!!
computing bag_of_words ......
computing bag_of_words done!
CPU times: user 1min 49s, sys: 8.14 s, total: 1min 57s
Wall time: 1min 57s


In [10]:
print(bag_of_words)

                    A.GC  A.GE  A.GF  A3900782  A4020252  A4172095  A4172097  \
10,2014-01-07,StC    NaN   NaN   NaN       NaN       NaN       NaN       NaN   
10,2014-01-09,StD    NaN   NaN   NaN       NaN       NaN       NaN       NaN   
10,2014-01-13,StC    NaN   NaN   NaN       NaN       NaN       NaN       NaN   
10,2014-01-14,StC    NaN   NaN   NaN       NaN       NaN       NaN       NaN   
10,2014-01-14,StD    NaN   NaN   NaN       NaN       NaN       NaN       NaN   
...                  ...   ...   ...       ...       ...       ...       ...   
996,2014-11-04,BfC   NaN   NaN   NaN       NaN       NaN       NaN       NaN   
996,2014-11-06,BfC   NaN   NaN   NaN       NaN       NaN       NaN       NaN   
996,2014-11-13,BfD   NaN   NaN   NaN       NaN       NaN       NaN       NaN   
996,2014-12-09,BfC   NaN   NaN   NaN       NaN       NaN       NaN       NaN   
999,2014-10-27,BfC   NaN   NaN   NaN       NaN       NaN       NaN       NaN   

                    AA.AA  AA.GI  AA.GL

In [11]:
print(bag_of_words.isna().sum().sum())

# vals = bag_of_words.values
# for i in range(10):
#     print(len(vals[i]))

3456142416


In [12]:
# volume
# (274150 * 11935) - 3266355180
# frac_out
(287800 * 12013) - 3456142416

1198984

In [ ]:
data['price'] = (data['ENTRD_VOL_QT'] * data['RPTD_PR'])/100

In [ ]:
data['price'].describe()

In [ ]:
"""Compute Dc_v4 which is count of bonds on given dealer and day seperated buy and sell"""
data['document_date'] = data['TRD_EXCTN_DTTM'].dt.date.apply(lambda x: str(x))
create_buy_document_no_source_vectorize = np.vectorize(create_buy_document_no_source)
create_sell_document_no_source_vectorize = np.vectorize(create_sell_document_no_source)
client_to_delete_vectorize = np.vectorize(client_to_delete)
print("creating documents ......")
# Add new column Dc_v4_S which is the string representation of report dealer buy on the specific day
data['Dc_v4_S'] = create_sell_document_no_source_vectorize(data['Report_Dealer_Index'].values,data['Contra_Party_Index'].values,data['document_date'].values)
# Add new column Dc_v4_B which is the string representation of report dealer sell on the specific day
data['Dc_v4_B'] = create_buy_document_no_source_vectorize(data['Report_Dealer_Index'].values,data['Contra_Party_Index'].values,data['document_date'].values)
print("documents created!!")

In [ ]:
type(data['Dc_v4_S'].iloc[7])

In [ ]:
1 != 2

In [ ]:
data_sub = data[['Dc_v4_S','Dc_v4_B','BOND_SYM_ID','price']].copy()
data_gb_sell = data_sub[data_sub['Dc_v4_S']!='nan'].groupby(by=['Dc_v4_S','BOND_SYM_ID'])
data_gb_buy = data_sub[data_sub['Dc_v4_B']!='nan'].groupby(by=['Dc_v4_B','BOND_SYM_ID'])

In [ ]:
#sell_matrix = data_gb_sell['price'].sum().unstack(fill_value=0)

In [ ]:
data_gb_sell['price'].sum().to_sparse().unstack(level=-1)

In [ ]:
sell_df = data_gb_sell['price'].sum().to_frame()

In [ ]:
sell_df.describe()

In [ ]:
sell_df['']

In [ ]:
bins=[0,10000,100000,1000000,10000000,sell_df[sell_df['price']>0]['price'].max()]
labels = [i for i in range(len(bins)-1)]
pd.cut(sell_df[sell_df['price']>0]['price'],bins=bins,labels=labels)

In [ ]:
pd.DataFrame(preprocessing.minmax_scale(sell_df[sell_df['price']>0],feature_range=(0, 5))).describe()

In [ ]:
sell_df.groupby(level=0).count().iloc[12::].describe()

In [ ]:
data_gb_sell['price'].sum().describe()

In [ ]:
from scipy.stats import mstats

In [ ]:
data['price_winsor'] = pd.Series(mstats.winsorize(data['price'].values,limits=[0.10,0.10]))

In [ ]:
pd.Series(preprocessing.minmax_scale(data['price_winsor'],feature_range=[0,5])).describe()

In [ ]:
data['price_winsor']